## **m.ClovaTalk 챗봇 구현**
- HyperClova X와 프롬프트 엔지니어링을 활용하여 구현"
- 사용자가 입력한 질문에 대해서 자세한 정보를 답해주는 대화형 챗봇

In [2]:
import requests
import json

class CompletionExecutor:
    def __init__(self, host, api_key, api_key_primary_val, request_id):
        self._host = host
        self._api_key = api_key
        self._api_key_primary_val = api_key_primary_val
        self._request_id = request_id

    def execute(self, completion_request):
        headers = {
            'X-NCP-CLOVASTUDIO-API-KEY': self._api_key,
            'X-NCP-APIGW-API-KEY': self._api_key_primary_val,
            'X-NCP-CLOVASTUDIO-REQUEST-ID': self._request_id,
            'Content-Type': 'application/json; charset=utf-8',
            'Accept': 'text/event-stream'
        }

        response_text = ""

        with requests.post(self._host + '/testapp/v1/chat-completions/HCX-003',
                           headers=headers, json=completion_request, stream=True) as r:
            for line in r.iter_lines():
                if line:
                    decoded_line = line.decode("utf-8")
                    if 'data:' in decoded_line:
                        try:
                            data = json.loads(decoded_line.split('data:', 1)[1])
                            if 'message' in data and 'content' in data['message']:
                                message_content = data['message']['content']
                                response_text += message_content
                        except json.JSONDecodeError:
                            continue

        # 중복된 줄 제거 및 결과 정리
        lines = response_text.split('. ')
        seen = set()
        unique_lines = []
        for line in lines:
            stripped_line = line.strip()
            if stripped_line and stripped_line not in seen:
                seen.add(stripped_line)
                unique_lines.append(stripped_line)

        # 결과를 다시 합쳐서 반환
        unique_response = '. '.join(unique_lines)
        if not unique_response.endswith('.'):
            unique_response += '.'

        return unique_response.strip()


if __name__ == '__main__':
    user_input = input("도우미 챗봇 m.ClovaTalk입니다. 궁금한 점을 편하게 질문해주세요: ")

    preset_text = [
        {"role": "system", "content": "- 사용자가 시장의 최신 동향을 이해하고, 더 현명한 투자 결정을 내릴 수 있도록 돕기 위해 네이버 경제 뉴스의 최신 데이터를 활용하여 경제, 주식 투자 관련 최신 트렌드와 이슈에 대해 신속하고 정확하게 정보를 제공합니다.\r\n- 복잡한 경제 이론이나 데이터도 신규 투자자가 이해하기 쉽도록 직관적인 설명을 제공하며, 주식 시장과 투자에 대한 기본 개념 용어 등을 쉽고 간단하게 설명하여 투자의 기초를 탄탄히 할 수 있도록 합니다.\r\n- 투자 시작 방법, 포트폴리오 관리 전략, 리스크 관리 방법 등 투자와 관련된 실질적이고 구체적인 조언을 제공하여 사용자가 실제 투자 상황에서 유용하게 활용할 수 있는 정보를 제공합니다.\r\n- 사용자의 질문이나 요구사항에 기반하여 개별적으로 맞춤형 답변을 제공합니다. 사용자의 투자 경험 수준, 관심 분야 등을 고려하여 더욱 적절하고 유용한 정보를 제공합니다.\r\n- 존댓말을 사용하여 대화하는 동안 친절하고 예의 바른 응대를 합니다.\r\n- 텍스트 설명뿐만 아니라, 필요에 따라 실시간 데이터 조회 등 다양한 방식으로 정보를 제공합니다.\n\n"},
        {"role": "user", "content": user_input}
    ]

    request_data = {
        'messages': preset_text,
        'topP': 0.8,
        'topK': 0,
        'maxTokens': 512,
        'temperature': 0.1,
        'repeatPenalty': 1.2,
        'stopBefore': [],
        'includeAiFilters': True,
        'seed': 0
    }

    completion_executor = CompletionExecutor(
        host='https://clovastudio.stream.ntruss.com',
        api_key='NTA0MjU2MWZlZTcxNDJiY3m5+690v88SuZgiqOMe0St8ZNXTJfjElAe7GoyTPYHh',
        api_key_primary_val='ru4tMCkXifqyJNWgponbZg3mYLQzPKdMLULd33uO',
        request_id='5fda8f3e-8a61-4206-8b26-b117f40769c0'
    )

    response = completion_executor.execute(request_data)
    print(response)

도우미 챗봇 m.ClovaTalk입니다. 궁금한 점을 편하게 질문해주세요:  P/E 비율에 대해서 설명해주세요.


P/E 비율은 주가수익비율(Price Earning Ratio)의 약자로, 주식 시장에서 기업의 가치를 평가하는 데 사용되는 중요한 지표 중 하나입니다.

P/E 비율은 기업의 주가를 주당순이익(EPS)으로 나눈 값으로, 기업이 1주당 얼마의 이익을 창출하고 있는지를 나타냅니다. 즉, P/E 비율이 높다는 것은 기업이 벌어들이는 이익에 비해 주가가 높게 평가되어 있다는 것을 의미하며, 반대로 P/E 비율이 낮다는 것은 기업의 이익에 비해 주가가 낮게 평가되어 있다는 것을 의미합니다.

P/E 비율은 기업의 성장성, 수익성, 위험성 등을 종합적으로 반영하고 있기 때문에, 주식 시장에서 기업의 가치를 평가하는 데 매우 유용한 지표로 활용됩니다.

일반적으로 P/E 비율은 국가, 산업, 기업 등에 따라 다르기 때문에, 이를 고려하여 기업의 가치를 평가해야 합니다. 또한, P/E 비율은 시간이 지남에 따라 변동하기 때문에, 주기적으로 확인하고 분석하는 것이 중요합니다.

P/E 비율은 주식 투자를 할 때 중요한 참고 자료 중 하나이지만, 절대적인 지표는 아닙니다. 주식 투자를 할 때는 P/E 비율뿐만 아니라, 기업의 재무 상태, 경영 전략, 시장 동향 등 다양한 요소를 고려하여 신중하게 결정해야 합니다.P/E 비율은 주가수익비율(Price Earning Ratio)의 약자로, 주식 시장에서 기업의 가치를 평가하는 데 사용되는 중요한 지표 중 하나입니다.

P/E 비율은 기업의 주가를 주당순이익(EPS)으로 나눈 값으로, 기업이 1주당 얼마의 이익을 창출하고 있는지를 나타냅니다. 주식 투자를 할 때는 P/E 비율뿐만 아니라, 기업의 재무 상태, 경영 전략, 시장 동향 등 다양한 요소를 고려하여 신중하게 결정해야 합니다.
